In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

In [ ]:

file_path1= "/Users/fanbu/Desktop/AI Agent for Time-Series Forecasting in Financial Planning/AI-Agent/data/India_cc_transactions.csv"

In [ ]:
data = pd.read_csv(file_path1)
# Make sure your column is called 'Date'
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')
data

In [ ]:
daily_card = data.groupby(['Date', 'Card Type'])['Amount'].sum().reset_index()


In [ ]:
daily_card.head()

In [ ]:
# how do the variables relate to each other?

In [ ]:
import numpy as np
import pandas as pd

def correlation_ratio(categories, values):
    """
    Calculates correlation ratio (eta) between categorical and continuous variables.
    Returns a value between 0 (no relationship) and 1 (perfect relationship).
    """
    fcat, _ = pd.factorize(categories)
    cat_means = np.array([values[fcat == i].mean() for i in range(len(np.unique(fcat)))])
    n = np.array([np.sum(fcat == i) for i in range(len(np.unique(fcat)))])
    grand_mean = values.mean()
    ss_between = np.sum(n * (cat_means - grand_mean)**2)
    ss_total = np.sum((values - grand_mean)**2)
    return np.sqrt(ss_between / ss_total)


In [ ]:
# Compute the correlation ratio for each categorical variable vs Amount
results = {}
for col in ['Card Type', 'Exp Type', 'Gender', 'City']:
    results[col] = correlation_ratio(data[col], data['Amount'])

# Convert to DataFrame for readability
corr_df = pd.DataFrame(list(results.items()), columns=['Variable', 'Correlation_with_Amount'])
corr_df = corr_df.sort_values('Correlation_with_Amount', ascending=False)
print(corr_df)


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))


In [ ]:
categorical_cols = ['Card Type', 'Exp Type', 'Gender', 'City']
n = len(categorical_cols)

# Initialize empty DataFrame
cramers_matrix = pd.DataFrame(np.zeros((n, n)), 
                              index=categorical_cols, 
                              columns=categorical_cols)

# Fill the matrix
for i in range(n):
    for j in range(n):
        if i == j:
            cramers_matrix.iloc[i,j] = 1.0  # a variable perfectly correlates with itself
        else:
            cramers_matrix.iloc[i,j] = cramers_v(data[categorical_cols[i]], data[categorical_cols[j]])

cramers_matrix


In [ ]:


# Create a 'Month' column
data['Month'] = data['Date'].dt.to_period('M')
# Aggregate transaction amounts per Month and Card Type
monthly_card = data.groupby(['Month', 'Card Type'])['Amount'].sum().reset_index()

# Convert Month back to datetime (optional, useful for plotting)
monthly_card['Month'] = monthly_card['Month'].dt.to_timestamp()

sns.lineplot(data=monthly_card, x='Month', y='Amount', hue='Card Type', marker='o')
plt.title('Monthly Transaction Amount per Card Type')
plt.ylabel('Total Amount')
plt.xlabel('Month')
plt.xticks(rotation=45)
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Ensure Date column is datetime
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Create a 'Month' column
data['Month'] = data['Date'].dt.to_period('M')

# Aggregate transaction amounts per Month and Exp Type
monthly_exp = data.groupby(['Month', 'Exp Type'])['Amount'].sum().reset_index()

# Convert Month back to datetime
monthly_exp['Month'] = monthly_exp['Month'].dt.to_timestamp()

# Create the line plot
plt.figure(figsize=(10,6))
ax = sns.lineplot(data=monthly_exp, x='Month', y='Amount', hue='Exp Type', marker='o')

# Add vertical lines for each January 1
years = monthly_exp['Month'].dt.year.unique()
for year in years:
    jan_first = pd.Timestamp(year=year, month=1, day=1)
    ax.axvline(jan_first, color='gray', linestyle='--', alpha=0.5)

# Customize labels and title
plt.title('Monthly Transaction Amount per Expense Type')
plt.ylabel('Total Amount')
plt.xlabel('Month')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:

# Pivot so Expense Types are columns
exp_pivot = monthly_exp.pivot(index='Month', columns='Exp Type', values='Amount').fillna(0)

In [ ]:
exp_pivot

In [ ]:
# Compute Pearson correlation between Expense Type trends
corr_matrix = exp_pivot.corr()
corr_matrix

In [ ]:
# bill, food and entertainment 
# Define mapping of Expense Types to new grouped segment
def group_expense(exp):
    if exp in ['Bills', 'Food', 'Entertainment']:
        return 'Everyday'
    else:
        return exp

# Apply mapping to create new column
data['Exp_Type_Group'] = data['Exp Type'].apply(group_expense)

# Check the result
data[['Exp Type', 'Exp_Type_Group']].drop_duplicates().sort_values('Exp_Type_Group')


In [ ]:
data

In [ ]:
# Sum Amount by city
city_totals = data.groupby('City')['Amount'].sum().reset_index()

# Sort descending
city_totals = city_totals.sort_values('Amount', ascending=False)
# Percentage of total
city_totals['Percentage'] = city_totals['Amount'] / city_totals['Amount'].sum() * 100

# Round for readability
city_totals['Percentage'] = city_totals['Percentage'].round(2)

city_totals.head(10)  # top 10 cities by total amount


In [ ]:
# 1. Compute total spending per city
city_totals = data.groupby('City')['Amount'].sum().reset_index()

# 2. Sort descending and get top 4 cities
top_cities = city_totals.sort_values('Amount', ascending=False).head(4)['City'].tolist()

# 3. Create a new column for City Segment
data['City_Segment'] = data['City'].apply(lambda x: x if x in top_cities else 'Other')

# 4. Check results
data[['City', 'City_Segment']].drop_duplicates().sort_values('City_Segment')


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Make sure Month is datetime
data['Month'] = pd.to_datetime(data['Date']).dt.to_period('M')

# Aggregate monthly spending by City_Segment
monthly_segment = data.groupby(['Month', 'City_Segment'])['Amount'].sum().reset_index()

# Convert Month back to datetime for plotting
monthly_segment['Month'] = monthly_segment['Month'].dt.to_timestamp()

# Plot line chart
plt.figure(figsize=(12,6))
sns.lineplot(
    data=monthly_segment,
    x='Month',
    y='Amount',
    hue='City_Segment',
    marker='o'
)
plt.title('Monthly Transaction Amount by City Segment')
plt.ylabel('Total Amount')
plt.xlabel('Month')
plt.xticks(rotation=45)
plt.legend(title='City Segment', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Make sure Month is datetime
data['Month'] = pd.to_datetime(data['Date']).dt.to_period('M')
monthly_city_expgroup = (
    data.groupby(['Month', 'City_Segment', 'Exp_Type_Group'])['Amount']
    .sum()
    .reset_index()
)
monthly_city_expgroup['Month'] = monthly_city_expgroup['Month'].dt.to_timestamp()

# Line plot
plt.figure(figsize=(14,6))
sns.lineplot(
    data=monthly_city_expgroup,
    x='Month',
    y='Amount',
    hue='Exp_Type_Group',
    style='City_Segment',
    markers=True,
    dashes=False
)

plt.title('Monthly Transaction Amount by City Segment and Expense Type Group')
plt.ylabel('Total Amount')
plt.xlabel('Month')
plt.xticks(rotation=45)
plt.legend(title='Expense Type Group / City Segment', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:

# Aggregate by Month, City_Segment, and Card Type
monthly_city_card = (
    data.groupby(['Month', 'City_Segment', 'Card Type'])['Amount']
    .sum()
    .reset_index()
)

# Convert Month back to datetime for plotting
monthly_city_card['Month'] = monthly_city_card['Month'].dt.to_timestamp()
# Pivot for stacked bar chart
pivot = monthly_city_card.pivot_table(
    index='Month', 
    columns=['City_Segment', 'Card Type'], 
    values='Amount', 
    fill_value=0
)

# Plot stacked bar chart
pivot.plot(kind='bar', stacked=True, figsize=(14,6))
plt.title('Monthly Transaction Amount by City Segment and Card Type')
plt.ylabel('Total Amount')
plt.xlabel('Month')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:


# Aggregate total spending by Card Type and Expense Type
card_exp = data.groupby(['Card Type', 'Exp Type'])['Amount'].sum().reset_index()

# Pivot the data so Expense Types become columns
card_exp_pivot = card_exp.pivot(index='Card Type', columns='Exp Type', values='Amount').fillna(0)

# Plot stacked bar chart
card_exp_pivot.plot(kind='bar', stacked=True, figsize=(10,6))

plt.title('Transaction Amount by Expense Type for Each Card Type')
plt.ylabel('Total Amount')
plt.xlabel('Card Type')
plt.legend(title='Expense Type', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
# card type is not a good predictor of amount

In [ ]:
# Average monthly spending per city
city_avg = data.groupby('City')['Amount'].mean().reset_index()

# Median split into High and Low
median_amount = city_avg['Amount'].median()
city_avg['City_Tier'] = city_avg['Amount'].apply(lambda x: 'High' if x >= median_amount else 'Low')

# Merge back into main data
data1 = data.merge(city_avg[['City', 'City_Tier']], on='City', how='left')


In [ ]:
data1

In [ ]:
# Create Month column
data1['Month'] = data1['Date'].dt.to_period('M')

# Aggregate spending by City_Tier × Exp Type
monthly_tier_exp = (
    data1.groupby(['Month', 'City_Tier', 'Exp Type'])['Amount']
    .sum()
    .reset_index()
)

# Convert Month to datetime
monthly_tier_exp['Month'] = monthly_tier_exp['Month'].dt.to_timestamp()

In [ ]:
sns.lineplot(data=monthly_tier_exp, x='Month', y='Amount', hue='Exp Type', style='City_Tier', markers=True)


In [ ]:

# Convert Month to datetime
monthly_tier_exp['Month'] = monthly_tier_exp['Month'].dt.to_timestamp()


In [ ]:
plt.figure(figsize=(10,6))
ax = sns.lineplot(
    data=monthly_tier,
    x='Month',
    y='Amount',
    hue='Spending Level',
    marker='o'
)

# Add vertical lines for each Jan 1
years = monthly_tier['Month'].dt.year.unique()
for year in years:
    ax.axvline(pd.Timestamp(year=year, month=1, day=1),
               color='gray', linestyle='--', alpha=0.5)

plt.title('Monthly Transaction Amount by City Spending Tier')
plt.ylabel('Total Amount')
plt.xlabel('Month')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Ensure Date column is datetime
data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# Create a 'Month' column
data['Month'] = data['Date'].dt.to_period('M')

# Aggregate transaction amounts per Month and Gender
monthly_gender = data.groupby(['Month', 'Gender'])['Amount'].sum().reset_index()

# Convert Month back to datetime (useful for plotting)
monthly_gender['Month'] = monthly_gender['Month'].dt.to_timestamp()

# Ensure continuous months for all genders
all_months = pd.date_range(start=monthly_gender['Month'].min(),
                           end=monthly_gender['Month'].max(), freq='MS')
all_combinations = pd.MultiIndex.from_product(
    [all_months, monthly_gender['Gender'].unique()],
    names=['Month', 'Gender']
)
monthly_gender = (
    monthly_gender
    .set_index(['Month', 'Gender'])
    .reindex(all_combinations, fill_value=0)
    .reset_index()
)

# Plot the results
plt.figure(figsize=(10,6))
ax = sns.lineplot(data=monthly_gender, x='Month', y='Amount', hue='Gender', marker='o')

# Add vertical lines for each January 1
years = monthly_gender['Month'].dt.year.unique()
for year in years:
    ax.axvline(pd.Timestamp(year=year, month=1, day=1),
               color='gray', linestyle='--', alpha=0.5)

# Customize chart
plt.title('Monthly Transaction Amount by Gender')
plt.ylabel('Total Amount')
plt.xlabel('Month')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing


In [ ]:

# Assume daily_card has columns: 'Date', 'Card Type', 'Amount'
# Step 1: Convert Date column to datetime
daily_card['Date'] = pd.to_datetime(daily_card['Date'], errors='coerce')

# Step 2: Drop rows with invalid dates
daily_card = daily_card.dropna(subset=['Date'])

# Step 3: Reindex to include all days per Card Type
all_card_types = daily_card['Card Type'].unique()
daily_card_full = pd.DataFrame()

for card in all_card_types:
    # Filter for this Card Type
    df_card = daily_card[daily_card['Card Type'] == card].set_index('Date').sort_index()
    
    # Create a complete daily index for the period
    all_days = pd.date_range(start=df_card.index.min(), end=df_card.index.max(), freq='D')
    df_card = df_card.reindex(all_days, fill_value=0)  # Fill missing days with 0
    
    # Add Card Type column back
    df_card['Card Type'] = card
    
    # Reset index
    df_card = df_card.rename_axis('Date').reset_index()
    
    # Append to full dataframe
    daily_card_full = pd.concat([daily_card_full, df_card], ignore_index=True)

# Step 4: Sort final dataframe
daily_card_full = daily_card_full.sort_values(['Card Type', 'Date']).reset_index(drop=True)

daily_card_full.head()


In [ ]:
daily_card_full.groupby("Card Type").count()

In [ ]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np


In [ ]:

# Step 1: Filter for Gold card
gold_card = daily_card_full[daily_card_full['Card Type'] == 'Gold'].copy()

# Step 2: Ensure Date is datetime
gold_card['Date'] = pd.to_datetime(gold_card['Date'], errors='coerce')
gold_card = gold_card.dropna(subset=['Date'])

# Step 3: Set Date as index and reindex to include all days
gold_card = gold_card.set_index('Date').sort_index()
all_days = pd.date_range(start=gold_card.index.min(), end=gold_card.index.max(), freq='D')
gold_card = gold_card.reindex(all_days, fill_value=0)
gold_card.index.name = 'Date'



In [ ]:
gold_card

In [ ]:
# Step 4: Split train/test
train_end = pd.Timestamp('2014-12-31')
train_df = gold_card[gold_card.index <= train_end]
test_df = gold_card[gold_card.index > train_end]


In [ ]:
# Step 5: Fit Exponential Smoothing on train set
model = ExponentialSmoothing(train_df['Amount'], trend='add', seasonal='add', seasonal_periods=7)
fit = model.fit()

# Step 6: Forecast for test period
forecast = fit.forecast(len(test_df))


In [ ]:



# Step 7: Plot
plt.figure(figsize=(12,4))
plt.plot(train_df.index, train_df['Amount'], label='Train')
plt.plot(test_df.index, test_df['Amount'], label='Test')
plt.plot(forecast.index, forecast, label='Forecast', linestyle='--')
plt.title('Exponential Smoothing Forecast - Gold Card')
plt.xlabel('Date')
plt.ylabel('Amount')
plt.legend()
plt.show()

# Step 8: Evaluate

rmse = np.sqrt(mean_squared_error(test_df['Amount'], forecast))
mae = mean_absolute_error(test_df['Amount'], forecast)
mape = np.mean(np.abs((test_df['Amount'] - forecast) / (test_df['Amount'] + 1e-8))) * 100  # avoid division by zero

print(f'Gold Card Forecast Performance:')
print(f'RMSE: {rmse:.2f}')
print(f'MAE:  {mae:.2f}')
print(f'MAPE: {mape:.2f}%')

# Model Improvement: Prophet

In [ ]:
from prophet import Prophet


In [ ]:


# Step 1: Filter for Gold card
gold_card_prophet = daily_card_full[daily_card_full['Card Type'] == 'Gold'].copy()

# Step 2: Ensure Date is datetime
gold_card_prophet['Date'] = pd.to_datetime(gold_card_prophet['Date'], errors='coerce')
gold_card_prophet = gold_card_prophet.dropna(subset=['Date'])

# Step 3: Aggregate daily and fill missing dates
gold_card_prophet = gold_card_prophet.groupby('Date', as_index=False)['Amount'].sum()
all_days_prophet = pd.date_range(start=gold_card_prophet['Date'].min(),
                                 end=gold_card_prophet['Date'].max(),
                                 freq='D')
gold_card_prophet = (
    gold_card_prophet
    .set_index('Date')
    .reindex(all_days_prophet, fill_value=0)
    .rename_axis('Date')
    .reset_index()
)

# Step 4: Rename for Prophet
gold_card_prophet = gold_card_prophet.rename(columns={'Date': 'ds', 'Amount': 'y'})

# Step 5: Train/test split
train_df_prophet = gold_card_prophet[gold_card_prophet['ds'] <= '2014-12-31']
test_df_prophet = gold_card_prophet[gold_card_prophet['ds'] > '2014-12-31']



In [ ]:

# Step 6: Fit Prophet model
model_prophet = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=False,
    changepoint_prior_scale=0.5  # tune if needed
)
model_prophet.fit(train_df_prophet)



In [ ]:
# ✅ Step 7: Create future dataframe that definitely covers test dates
future_prophet = model_prophet.make_future_dataframe(
    periods=len(test_df_prophet), 
    freq='D',
    include_history=True
)
forecast_prophet = model_prophet.predict(future_prophet)

# Step 8: Align forecast with test data
forecast_test_prophet = forecast_prophet[
    forecast_prophet['ds'].between(test_df_prophet['ds'].min(), test_df_prophet['ds'].max())
][['ds', 'yhat']]

# Step 9: Merge with actuals
results_prophet = test_df_prophet.merge(forecast_test_prophet, on='ds', how='inner')

# ✅ Check: Should not be empty now
print(f"Forecast rows: {len(results_prophet)}")

In [ ]:
# Step 11: Plot
plt.figure(figsize=(12,4))
plt.plot(train_df_prophet['ds'], train_df_prophet['y'], label='Train')
plt.plot(test_df_prophet['ds'], test_df_prophet['y'], label='Test')
plt.plot(results_prophet['ds'], results_prophet['yhat'], label='Forecast', linestyle='--')
plt.title('Prophet Forecast - Gold Card')
plt.xlabel('Date')
plt.ylabel('Amount')
plt.legend()
plt.show()

# Optional: Prophet's built-in components plot
model_prophet.plot_components(forecast_prophet)
plt.show()

In [ ]:
# ✅ Check: Should not be empty now
print(f"Forecast rows: {len(results_prophet)}")

# Step 10: Evaluate
if len(results_prophet) > 0:
    rmse_prophet = np.sqrt(mean_squared_error(results_prophet['y'], results_prophet['yhat']))
    mae_prophet = mean_absolute_error(results_prophet['y'], results_prophet['yhat'])
    mape_prophet = np.mean(np.abs((results_prophet['y'] - results_prophet['yhat']) / (results_prophet['y'] + 1e-8))) * 100

    print(f"Gold Card - Prophet Forecast Performance:")
    print(f"RMSE: {rmse_prophet:.2f}")
    print(f"MAE:  {mae_prophet:.2f}")
    print(f"MAPE: {mape_prophet:.2f}%")
else:
    print("⚠️ No overlapping forecast and test dates found — check date ranges!")


## 20 segments models

In [ ]:
# Train: before 2015-01-01
train = monthly_segment[monthly_segment['Month'] < '2015-01-01']

# Test: from 2015-01-01 onward
test = monthly_segment[monthly_segment['Month'] >= '2015-01-01']

print(f"Train shape: {train.shape}, Test shape: {test.shape}")


In [ ]:
train_segment = train[(train['City_Segment']=='High') & (train['Exp_Type_Group']=='Bill_Food_Entertainment')]
test_segment = test[(test['City_Segment']=='High') & (test['Exp_Type_Group']=='Bill_Food_Entertainment')]

print(train_segment.head())
print(test_segment.head())


In [ ]:
# Sort for rolling operation
monthly_segment = monthly_segment.sort_values(['City_Segment', 'Exp_Type_Group', 'Month'])

# Compute rolling mean per segment
monthly_segment['Moving_Avg'] = (
    monthly_segment
    .groupby(['City_Segment', 'Exp_Type_Group'])['Amount']
    .transform(lambda x: x.rolling(window=3, min_periods=1).mean())
)

monthly_segment.head(10)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(14,7))
sns.lineplot(
    data=monthly_segment,
    x='Month',
    y='Moving_Avg',
    hue='Exp_Type_Group',
    style='City_Segment',
    markers=True,
    dashes=False
)

plt.title('3-Month Moving Average of Monthly Spending by Segment')
plt.ylabel('Amount')
plt.xlabel('Month')
plt.xticks(rotation=45)
plt.legend(title='Expense Type Group / City Segment', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()
